In [1]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import scipy
import sklearn
import random as rd
from IPython.display import display
import os 
from pathlib import Path
import shutil 


### IF WE HAVE THE CK+ DATABASE, WE CAN DO THIS CODE IN THE NEXTS CELLS TO CREATE A MINI DATABASE TO START WITH THE FEATURES

In [3]:
 len(os.listdir('../../ck+/Landmarks' ))

123

##Elegimos la cantidad de sujetos para la extraccion (=11) de ck al azar. Luego eligiremos una con interseccion 0 para la evaluacio (y una parte mas chica con interseccion 0 con las otras 2 papra validacion)

In [7]:
Emotion_subjects = os.listdir('../../ck+/Emotion_labels/Emotion' )
len(Emotion_subjects)
images_subjects = os.listdir('../../ck+/extended-cohn-kanade-images/cohn-kanade-images' )
len(images_subjects)
facts_subjects =  os.listdir('../../ck+/FACS_labels/FACS' )
len(facts_subjects)
landmarks_subjects =  os.listdir('../../ck+/Landmarks' )
len(landmarks_subjects)



['S125',
 'S133',
 'S078',
 'S117',
 'S029',
 'S035',
 'S109',
 'S079',
 'S076',
 'S099',
 'S051',
 'S156',
 'S148',
 'S506',
 'S045',
 'S077',
 'S502',
 'S052',
 'S090',
 'S014',
 'S095',
 'S104',
 'S083',
 'S130',
 'S127',
 'S059',
 'S151',
 'S136',
 'S073',
 'S085',
 'S056',
 'S069',
 'S114',
 'S138',
 'S071',
 'S137',
 'S067',
 'S110',
 'S091',
 'S503',
 'S096',
 'S070',
 'S121',
 'S046',
 'S158',
 'S139',
 'S097',
 'S010',
 'S058',
 'S037',
 'S092',
 'S102',
 'S120',
 'S504',
 'S063',
 'S106',
 'S061',
 'S066',
 'S086',
 'S107',
 'S135',
 'S055',
 'S112',
 'S501',
 'S074',
 'S131',
 'S084',
 'S101',
 'S054',
 'S072',
 'S134',
 'S126',
 'S089',
 'S999',
 'S155',
 'S119',
 'S064',
 'S129',
 'S103',
 'S124',
 'S080',
 'S075',
 'S154',
 'S062',
 'S105',
 'S113',
 'S111',
 'S082',
 'S093',
 'S087',
 'S053',
 'S088',
 'S147',
 'S034',
 'S060',
 'S065',
 'S081',
 'S042',
 'S032',
 'S108',
 'S160',
 'S005',
 'S057',
 'S115',
 'S157',
 'S022',
 'S132',
 'S116',
 'S044',
 'S149',
 'S026',
 

In [5]:

#dir is not keyword
def makemydir(whatever):
  try:
    os.makedirs(whatever)
  except OSError:
    pass
  # let exception propagate if we just can't
  # cd into the specified directory
  os.chdir(whatever)
    
makemydir()

In [4]:
def do_assigned_dir(orsrc, ordst, subjects):
    for number in subjects:
        src = orsrc + number
        dst = ordst + number
        destination = shutil.copytree(src, dst)
        print(destination)
    return 0

In [8]:
do_assigned_dir('../../ck+/extended-cohn-kanade-images/cohn-kanade-images/', '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/', images_subjects )
do_assigned_dir('../../ck+/Emotion_labels/Emotion/', '../ck+copy/Emotion_labels/Emotion/', Emotion_subjects )
do_assigned_dir('../../ck+/FACS_labels/FACS/', '../ck+copy/FACS_labels/FACS/', facts_subjects )
do_assigned_dir('../../ck+/Landmarks/', '../ck+copy/Landmarks/', landmarks_subjects )

../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S125
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S133
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S078
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S117
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S029
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S035
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S109
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S079
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S076
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S099
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S051
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S156
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S148
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S506
../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/S045
../ck+copy/extended-cohn-kanade-images/cohn-kanade-imag

../ck+copy/Emotion_labels/Emotion/S090
../ck+copy/Emotion_labels/Emotion/S014
../ck+copy/Emotion_labels/Emotion/S095
../ck+copy/Emotion_labels/Emotion/S104
../ck+copy/Emotion_labels/Emotion/S083
../ck+copy/Emotion_labels/Emotion/S130
../ck+copy/Emotion_labels/Emotion/S127
../ck+copy/Emotion_labels/Emotion/S059
../ck+copy/Emotion_labels/Emotion/S151
../ck+copy/Emotion_labels/Emotion/S136
../ck+copy/Emotion_labels/Emotion/S073
../ck+copy/Emotion_labels/Emotion/S085
../ck+copy/Emotion_labels/Emotion/S056
../ck+copy/Emotion_labels/Emotion/S069
../ck+copy/Emotion_labels/Emotion/S114
../ck+copy/Emotion_labels/Emotion/S138
../ck+copy/Emotion_labels/Emotion/S071
../ck+copy/Emotion_labels/Emotion/S137
../ck+copy/Emotion_labels/Emotion/S067
../ck+copy/Emotion_labels/Emotion/S110
../ck+copy/Emotion_labels/Emotion/S091
../ck+copy/Emotion_labels/Emotion/S503
../ck+copy/Emotion_labels/Emotion/S096
../ck+copy/Emotion_labels/Emotion/S070
../ck+copy/Emotion_labels/Emotion/S121
../ck+copy/Emotion_labels

../ck+copy/Landmarks/S078
../ck+copy/Landmarks/S117
../ck+copy/Landmarks/S029
../ck+copy/Landmarks/S035
../ck+copy/Landmarks/S109
../ck+copy/Landmarks/S079
../ck+copy/Landmarks/S076
../ck+copy/Landmarks/S099
../ck+copy/Landmarks/S051
../ck+copy/Landmarks/S156
../ck+copy/Landmarks/S148
../ck+copy/Landmarks/S506
../ck+copy/Landmarks/S045
../ck+copy/Landmarks/S077
../ck+copy/Landmarks/S502
../ck+copy/Landmarks/S052
../ck+copy/Landmarks/S090
../ck+copy/Landmarks/S014
../ck+copy/Landmarks/S095
../ck+copy/Landmarks/S104
../ck+copy/Landmarks/S083
../ck+copy/Landmarks/S130
../ck+copy/Landmarks/S127
../ck+copy/Landmarks/S059
../ck+copy/Landmarks/S151
../ck+copy/Landmarks/S136
../ck+copy/Landmarks/S073
../ck+copy/Landmarks/S085
../ck+copy/Landmarks/S056
../ck+copy/Landmarks/S069
../ck+copy/Landmarks/S114
../ck+copy/Landmarks/S138
../ck+copy/Landmarks/S071
../ck+copy/Landmarks/S137
../ck+copy/Landmarks/S067
../ck+copy/Landmarks/S110
../ck+copy/Landmarks/S091
../ck+copy/Landmarks/S503
../ck+copy/L

0

#### Aqui sacamos las carpetas que no tienen emocion definida o es contempt, que no usamos

In [9]:
deletefolders = []
for number in Emotion_subjects:
  
    src ='../ck+copy/Emotion_labels/Emotion/'  + number
    emotion_folders = np.sort(os.listdir(src))
    for number_emo in emotion_folders:
        emotion = src  +'/'+ number_emo             
        if not os.listdir(emotion):
            deletefolders.append([number, number_emo])
        else:
            filelabel = os.listdir(emotion)[0]
            with open(emotion+ '/'+filelabel) as f:
                line = float(f.readline())
            if (line ==2.0):
                deletefolders.append([number,number_emo])
                # 2 == contempt emotion
            
   

In [10]:
deletefolders

[['S125', '002'],
 ['S125', '003'],
 ['S125', '004'],
 ['S133', '001'],
 ['S133', '002'],
 ['S133', '004'],
 ['S078', '002'],
 ['S078', '003'],
 ['S078', '005'],
 ['S078', '006'],
 ['S117', '002'],
 ['S117', '004'],
 ['S117', '005'],
 ['S117', '007'],
 ['S035', '002'],
 ['S035', '003'],
 ['S035', '004'],
 ['S109', '001'],
 ['S109', '002'],
 ['S109', '004'],
 ['S079', '003'],
 ['S076', '002'],
 ['S076', '003'],
 ['S076', '004'],
 ['S099', '002'],
 ['S099', '003'],
 ['S051', '001'],
 ['S156', '002'],
 ['S148', '002'],
 ['S506', '002'],
 ['S045', '001'],
 ['S045', '002'],
 ['S045', '003'],
 ['S077', '002'],
 ['S077', '003'],
 ['S077', '004'],
 ['S077', '005'],
 ['S077', '007'],
 ['S502', '002'],
 ['S052', '002'],
 ['S052', '003'],
 ['S014', '004'],
 ['S095', '002'],
 ['S095', '003'],
 ['S095', '004'],
 ['S095', '005'],
 ['S104', '001'],
 ['S104', '002'],
 ['S104', '003'],
 ['S083', '001'],
 ['S083', '002'],
 ['S083', '004'],
 ['S130', '002'],
 ['S130', '003'],
 ['S130', '004'],
 ['S130', 

In [11]:
for folder in deletefolders:
    image = '../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/'+ folder[0]+'/' + folder[1]
    facs =  '../ck+copy/FACS_labels/FACS/'+ folder[0]+'/' + folder[1]
    emotion = '../ck+copy/Emotion_labels/Emotion/' + folder[0]+ '/' + folder[1]
    landmark = '../ck+copy/Landmarks/' + folder[0]+ '/' + folder[1]
    shutil.rmtree(image)
    shutil.rmtree(facs)
    shutil.rmtree(emotion)
    shutil.rmtree(landmark)

In [25]:
def percentage(percent, whole):
  return int((percent * whole) / 100.0)

percentage(60,123)

73

In [27]:
def choose_random_subjects(main_dir, local_dir, amount):
    subjects = os.listdir(main_dir + local_dir )
    random_subjects = np.random.choice(subjects, amount,replace=False )
    
    return random_subjects
    
subject_number =choose_random_subjects('../../ck+/','extended-cohn-kanade-images/cohn-kanade-images', percentage(60, len(Emotion_subjects)))

len(subject_number)

73

In [29]:
def moveto_assigned_dir(orsrc, ordst, subjects):
    for number in subjects:
        src = orsrc + number
        dst = ordst + number
        destination = shutil.move(src, dst)
        print(destination)
    return 0

In [30]:
moveto_assigned_dir('../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/', '../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/', subject_number )
moveto_assigned_dir('../ck+copy/Emotion_labels/Emotion/', '../ck+copytraining/Emotion_labels/Emotion/' , subject_number )
moveto_assigned_dir('../ck+copy/FACS_labels/FACS/', '../ck+copytraining/FACS_labels/FACS/' , subject_number )
moveto_assigned_dir('../ck+copy/Landmarks/', '../ck+copytraining/Landmarks/', subject_number )
  

../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S022
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S074
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S106
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S155
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S037
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S103
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S121
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S090
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S065
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S134
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S895
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S137
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S067
../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/S131
../ck+

0

In [31]:
len(os.listdir('../ck+copy/extended-cohn-kanade-images/cohn-kanade-images/'))

50

In [32]:
len(os.listdir('../ck+copytraining/extended-cohn-kanade-images/cohn-kanade-images/'))

73

In [127]:
subjects_landmarks = '../ck+copy/Landmarks/'
sorted_subjects =   np.sort(os.listdir('../ck+copy/Landmarks/'))  
one_subject_landmarks = '../ck+copy/Landmarks/' + 'S064/'
sorted_list = np.sort(os.listdir('../ck+copy/Landmarks/' + 'S064/')) #creo que este sort se puede quitar


    
distances_per_subject = []
for emotion in sorted_list:
    matrix_landmark_per_frame = []
    subject_landmark_emotion = one_subject_landmarks + str(emotion)    
    for i in range(0, len(os.listdir(subject_landmark_emotion))):
        data = pd.read_csv('../ck+copy/Landmarks/' + 'S064/' + str(emotion)+'/'+'S064_'+str(emotion)+'_'+ str(i+1).zfill(8)+'_landmarks.txt',  header=None)
        data.columns =['landmarks']
        quantity_of_landmarks = data.size #68
        data = data['landmarks'].str.split('   ', expand=True)
        data.columns =['landmarks', 'landmarkX', 'landmarkY']
        data =data.drop(columns=['landmarks'])
        landmarkXY=data['landmarkX'].append(data['landmarkY'], ignore_index=True)
        matrix_landmark_per_frame.append(landmarkXY_array)
    frame0 = matrix_landmark_per_frame[0]
    distance_landmark_per_frame = matrix_landmark_per_frame
    for i in range(0, len(distance_landmark_per_frame)):
        distance_landmark_per_frame[i] = distance_landmark_per_frame[i] - frame0
    distances_per_subject.append(distance_landmark_per_frame)
